## Import

In [112]:
import seaborn as sns;
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as TTS
from sklearn.model_selection import cross_val_score;
from sklearn.preprocessing import LabelEncoder;

# pipeline elements
from sklearn.decomposition import PCA # PCA = Principal Component Analysis
from sklearn.neighbors import KNeighborsClassifier as KNN 

# pipeline materiaux
from sklearn.pipeline import Pipeline # PCA = Principal Component Analysis
from sklearn.model_selection import GridSearchCV

## Collecte de données

In [59]:
# charger le dataset
fd = pd.read_csv('data/Fraud_Data.csv')
ip = pd.read_csv('data/IpAddress_to_Country.csv')

In [60]:
fd.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [61]:
ip.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [62]:
ip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138846 entries, 0 to 138845
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   lower_bound_ip_address  138846 non-null  float64
 1   upper_bound_ip_address  138846 non-null  int64  
 2   country                 138846 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 3.2+ MB


In [63]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         151112 non-null  int64  
 1   signup_time     151112 non-null  object 
 2   purchase_time   151112 non-null  object 
 3   purchase_value  151112 non-null  int64  
 4   device_id       151112 non-null  object 
 5   source          151112 non-null  object 
 6   browser         151112 non-null  object 
 7   sex             151112 non-null  object 
 8   age             151112 non-null  int64  
 9   ip_address      151112 non-null  float64
 10  class           151112 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 12.7+ MB


In [64]:
fd.dropna()
ip.dropna()
#ip.query("(country == 'China')")

,lower_bound_ip_address,upper_bound_ip_address,country
0,1.677722e+07,16777471,Australia
1,1.677747e+07,16777727,China
2,1.677773e+07,16778239,China
3,1.677824e+07,16779263,Australia
4,1.677926e+07,16781311,China
...,...,...,...
138841,3.758092e+09,3758093311,Hong Kong
138842,3.758093e+09,3758094335,India
138843,3.758095e+09,3758095871,China
138844,3.758096e+09,3758096127,Singapore


In [65]:
# On va parcourir le nombre de ligne contenu dans la dataset fd
# Si l'adresse ip contenu dans la data ip se trouve dans une des plages d'ip de la dataset de fd,
# stocke la ligne dans une variable. Si la ligne est bonne, créé des variable et les ajoute les colonnes en plus 
# des lignes correspondants dans fd.
#
# var: 
#         ligne: va stocker la ligne de l'ip se situant dans la plage d'adresse ip qui nous interesse
#         cou: va stocker le pays par rapport à la l'adresse ip récupéré
#         low: va stocker le lower_bound_ip_address par rapport à la l'adresse ip récupéré
#         up: va stocker le upper_bound_ip_address par rapport à la l'adresse ip récupéré


# Créer une list de la taille de fd
country = len(fd) * [0]                
lower_bound_ip_address = len(fd) * [0]
upper_bound_ip_address = len(fd) * [0]

for ind, row in fd.iterrows():          # Il va parcourir le nombre de ligne contenu dans fd
    ligne = ip[(ip['lower_bound_ip_address'] < row['ip_address']) &              
           (ip['upper_bound_ip_address'] > row['ip_address'])]       
    cou = ligne['country']  
    low = ligne['lower_bound_ip_address']  
    up = ligne['upper_bound_ip_address']  
    
    if len(cou) == 1:                   
        country[ind] = cou.values[0]  
        lower_bound_ip_address[ind] = low.values[0]  
        upper_bound_ip_address[ind] = up.values[0]

fd['country'] = country
fd['lower_bound_ip_address'] = lower_bound_ip_address
fd['upper_bound_ip_address'] = upper_bound_ip_address

In [82]:
fd.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,lower_bound_ip_address,upper_bound_ip_address
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,7.298089e+08,734003199
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,3.355443e+08,352321535
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,2.621440e+09,2621505535
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,0,0.000000e+00,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,4.154982e+08,415629311


In [78]:
fd['ip_address'].max()

4294850499.6788397

In [79]:
fd['upper_bound_ip_address'].max()

3758063615

In [80]:
fd.to_csv('data/IpAddress_to_Country_Fraud_Data.csv') # On créé un fichier csv avec toutes nos données

In [96]:
fd_ip = pd.read_csv('data/IpAddress_to_Country_Fraud_Data.csv')

In [97]:
fd_ip = fd_ip[fd_ip.country != '0']

In [100]:
fd_ip

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,lower_bound_ip_address,upper_bound_ip_address
0,0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,7.298089e+08,734003199
1,1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,3.355443e+08,352321535
2,2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,2.621440e+09,2621505535
4,4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,4.154982e+08,415629311
5,5,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0,Canada,2.809266e+09,2809331711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151106,151106,360761,2015-02-10 06:39:44,2015-06-03 08:18:30,13,EFCFFMUKFRDHJ,SEO,Safari,F,42,8.740657e+08,0,United States,8.724152e+08,889192447
151107,151107,345170,2015-01-27 03:03:34,2015-03-29 00:30:47,43,XPSKTWGPWINLR,SEO,Chrome,M,28,3.451155e+09,1,United States,3.451126e+09,3451387903
151108,151108,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,LYSFABUCPCGBA,SEO,Safari,M,32,2.439047e+09,0,Netherlands,2.436039e+09,2441084927
151109,151109,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,MEQHCSJUBRBFE,SEO,IE,F,26,2.748471e+09,0,Japan,2.748449e+09,2748514303


In [103]:
Y = fd_ip['class'].astype('category').cat.codes # On veut afficher le type de fleur
X = fd_ip.drop('class', axis='columns') # On garde que les dimension de la fleur

In [120]:
Y = LabelEncoder().fit_transform(Y)

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,country,lower_bound_ip_address,upper_bound_ip_address
0,0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,Japan,7.298089e+08,734003199
1,1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,United States,3.355443e+08,352321535
2,2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,United States,2.621440e+09,2621505535
4,4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,United States,4.154982e+08,415629311
5,5,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,Canada,2.809266e+09,2809331711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151106,151106,360761,2015-02-10 06:39:44,2015-06-03 08:18:30,13,EFCFFMUKFRDHJ,SEO,Safari,F,42,8.740657e+08,United States,8.724152e+08,889192447
151107,151107,345170,2015-01-27 03:03:34,2015-03-29 00:30:47,43,XPSKTWGPWINLR,SEO,Chrome,M,28,3.451155e+09,United States,3.451126e+09,3451387903
151108,151108,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,LYSFABUCPCGBA,SEO,Safari,M,32,2.439047e+09,Netherlands,2.436039e+09,2441084927
151109,151109,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,MEQHCSJUBRBFE,SEO,IE,F,26,2.748471e+09,Japan,2.748449e+09,2748514303


In [121]:
X['source'] = LabelEncoder().fit_transform(X['source'])
X['browser'] = LabelEncoder().fit_transform(X['browser'])
X['sex'] = LabelEncoder().fit_transform(X['sex'])
X['country'] = LabelEncoder().fit_transform(X['country'])

In [127]:
X

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,country,lower_bound_ip_address,upper_bound_ip_address
0,0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,2,0,1,39,7.327584e+08,84,7.298089e+08,734003199
1,1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,0,0,0,53,3.503114e+08,171,3.355443e+08,352321535
2,2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,2,3,1,53,2.621474e+09,171,2.621440e+09,2621505535
4,4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,0,4,1,45,4.155831e+08,171,4.154982e+08,415629311
5,5,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,0,0,1,18,2.809315e+09,32,2.809266e+09,2809331711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151106,151106,360761,2015-02-10 06:39:44,2015-06-03 08:18:30,13,EFCFFMUKFRDHJ,2,4,0,42,8.740657e+08,171,8.724152e+08,889192447
151107,151107,345170,2015-01-27 03:03:34,2015-03-29 00:30:47,43,XPSKTWGPWINLR,2,0,1,28,3.451155e+09,171,3.451126e+09,3451387903
151108,151108,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,LYSFABUCPCGBA,2,4,1,32,2.439047e+09,118,2.436039e+09,2441084927
151109,151109,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,MEQHCSJUBRBFE,2,2,0,26,2.748471e+09,84,2.748449e+09,2748514303


In [126]:
X = X.drop(['device_id'],axis=1)
X = X.drop(['signup_time'],axis=1)
X = X.drop(['purchase_time'],axis=1)

X.info()

KeyError: "['device_id'] not found in axis"